## Organograma

Evolução história dos vencimentos dos servidores

### Data Sources
- file1 : Description of where this file came from

### Changes
- 07-01-2023 : Started project

In [1]:
import pandas as pd
from pathlib import Path
from datetime import datetime
import chardet

### File Locations

In [2]:
today = datetime.today()
raw_dir = Path.cwd() / "data" / "raw"
summary_file = Path.cwd() / "data" / "processed" / f"summary_file.pkl"
processed_dir = Path.cwd() / "data" / "processed" 

In [3]:
lista_folhas_xlsx = list(raw_dir.glob('*.xlsx'))
print('Qtde arquivos encontrados', len([f.name for f in lista_folhas_xlsx]))
print('Nome primeiro arquivo:', ([f.name for f in lista_folhas_xlsx])[0])
print('Nome último arquivo:', ([f.name for f in lista_folhas_xlsx])[-1])

Qtde arquivos encontrados 73
Nome primeiro arquivo: 2017-07.xlsx
Nome último arquivo: 2023-07.xlsx


In [4]:
with lista_folhas_xlsx[0].open(mode='rb') as file:
    print(chardet.detect(file.read()))

{'encoding': None, 'confidence': 0.0, 'language': None}


In [5]:
dtypes = {
    'Matrícula' : str,
}

df_dict ={f"df_{xlsx.name.split('.')[0].replace('-', '_')}": pd.read_excel(xlsx, skiprows=11, dtype=dtypes) for xlsx in lista_folhas_xlsx}

In [7]:
df_dict['df_2017_07'].head(5)

,Matrícula,Nome,Secretaria,Local de Trabalho,Cargo,Escolaridade,Tempo de Casa,Data Admissão,Data Demissão,Tipo Folha,Condição,Fun_Id,Tpf_Id,Cnd_Id,Salário Bruto,Salário Líquido
0,000028,RAIMUNDO ANTONIO LUIZ,Secretaria Geral Legislativa,Departamento Financeiro,Chefe de Divisço,NaN,39 ano(s),28/01/1982,03/11/2021,Adiantamento,Ativo,28,1,1,3300.09,3300
1,000028,RAIMUNDO ANTONIO LUIZ,Secretaria Geral Legislativa,Departamento Financeiro,Chefe de Divisço,NaN,39 ano(s),28/01/1982,03/11/2021,Pagamento Mensal,Ativo,28,6,1,19416.31,3361
2,000091,PEDRO HIDEKI KOMURA,Vereadores,Inativos,Presidente da Camara,NaN,30 ano(s),01/01/1993,NaN,Pagamento Mensal,Sem Remunera‡Æo Fixa,91,6,2,12959.15,7840
3,000148,JOSE ROBERTO RIBEIRO,Secretaria Geral Administrativa,Departamento Administrativo,Motorista,NaN,26 ano(s),03/07/1995,01/10/2021,Pagamento Mensal,Ativo,148,6,1,4721.68,2333
4,000148,JOSE ROBERTO RIBEIRO,Secretaria Geral Administrativa,Departamento Administrativo,Motorista,NaN,26 ano(s),03/07/1995,01/10/2021,Adiantamento,Ativo,148,1,1,1216.33,1216


### Column Cleanup

- Remove all leading and trailing spaces
- Rename the columns for consistency.

In [8]:
# https://stackoverflow.com/questions/30763351/removing-space-in-dataframe-python
for df in df_dict.values():
    df.columns = [x.strip() for x in df.columns]
{col: '' for col in df_dict[list(df_dict.keys())[0]].columns}

{'Matrícula': '',
 'Nome': '',
 'Secretaria': '',
 'Local de Trabalho': '',
 'Cargo': '',
 'Escolaridade': '',
 'Tempo de Casa': '',
 'Data Admissão': '',
 'Data Demissão': '',
 'Tipo Folha': '',
 'Condição': '',
 'Fun_Id': '',
 'Tpf_Id': '',
 'Cnd_Id': '',
 'Salário Bruto': '',
 'Salário Líquido': ''}

In [9]:
cols_to_rename = {
    'Matrícula': 'matricula',
    'Nome': 'nome',
    'Secretaria': 'secretaria',
    'Local de Trabalho': 'local',
    'Cargo': 'cargo',
    'Escolaridade': 'escolaridade',
    'Tempo de Casa': 'tempo',
    'Data Admissão': 'admissao',
    'Data Demissão': 'demissao',
    'Tipo Folha': 'tipo_folha',
    'Condição': 'condicao',
    'Fun_Id': 'id_matricula',
    'Salário Bruto': 'valor_bruto',
    'Salário Líquido': 'valor_liquido'
}


for df in df_dict.values():
    df.rename(columns=cols_to_rename, inplace=True)

In [10]:
from pprint import pprint

contador_dict = dict()
for df in df_dict.values():
    for col in df.columns:
        contador_dict.setdefault(col, 0)
        contador_dict[col] += 1
pprint(contador_dict)

{'Cnd_Id': 73,
 'Tpf_Id': 73,
 'admissao': 73,
 'cargo': 73,
 'condicao': 73,
 'demissao': 73,
 'escolaridade': 73,
 'id_matricula': 73,
 'local': 73,
 'matricula': 73,
 'nome': 73,
 'secretaria': 73,
 'tempo': 73,
 'tipo_folha': 73,
 'valor_bruto': 73,
 'valor_liquido': 73}


### Clean Up Data Types

In [11]:
df.dtypes

matricula         object
nome              object
secretaria        object
local             object
cargo             object
escolaridade     float64
tempo             object
admissao          object
demissao         float64
tipo_folha        object
condicao          object
id_matricula       int64
Tpf_Id             int64
Cnd_Id             int64
valor_bruto      float64
valor_liquido      int64
dtype: object

In [12]:
for nome, df in df_dict.items():
    _, df['ano'], df['mes'] = nome.split('_')
    df['competencia'] = f"{nome.split('_')[1]}-{nome.split('_')[-1]}"

df_dict[list(df_dict.keys())[0]].head(3)

,matricula,nome,secretaria,local,cargo,escolaridade,tempo,admissao,demissao,tipo_folha,condicao,id_matricula,Tpf_Id,Cnd_Id,valor_bruto,valor_liquido,ano,mes,competencia
0,000028,RAIMUNDO ANTONIO LUIZ,Secretaria Geral Legislativa,Departamento Financeiro,Chefe de Divisço,NaN,39 ano(s),28/01/1982,03/11/2021,Adiantamento,Ativo,28,1,1,3300.09,3300,2017,07,2017-07
1,000028,RAIMUNDO ANTONIO LUIZ,Secretaria Geral Legislativa,Departamento Financeiro,Chefe de Divisço,NaN,39 ano(s),28/01/1982,03/11/2021,Pagamento Mensal,Ativo,28,6,1,19416.31,3361,2017,07,2017-07
2,000091,PEDRO HIDEKI KOMURA,Vereadores,Inativos,Presidente da Camara,NaN,30 ano(s),01/01/1993,NaN,Pagamento Mensal,Sem Remunera‡Æo Fixa,91,6,2,12959.15,7840,2017,07,2017-07


In [13]:
# Concatena todos os dataframes
df = pd.concat(df_dict.values())
print('df.shape', df.shape)
df.head()


df.shape (30024, 19)


,matricula,nome,secretaria,local,cargo,escolaridade,tempo,admissao,demissao,tipo_folha,condicao,id_matricula,Tpf_Id,Cnd_Id,valor_bruto,valor_liquido,ano,mes,competencia
0,000028,RAIMUNDO ANTONIO LUIZ,Secretaria Geral Legislativa,Departamento Financeiro,Chefe de Divisço,NaN,39 ano(s),28/01/1982,03/11/2021,Adiantamento,Ativo,28,1,1,3300.09,3300,2017,07,2017-07
1,000028,RAIMUNDO ANTONIO LUIZ,Secretaria Geral Legislativa,Departamento Financeiro,Chefe de Divisço,NaN,39 ano(s),28/01/1982,03/11/2021,Pagamento Mensal,Ativo,28,6,1,19416.31,3361,2017,07,2017-07
2,000091,PEDRO HIDEKI KOMURA,Vereadores,Inativos,Presidente da Camara,NaN,30 ano(s),01/01/1993,NaN,Pagamento Mensal,Sem Remunera‡Æo Fixa,91,6,2,12959.15,7840,2017,07,2017-07
3,000148,JOSE ROBERTO RIBEIRO,Secretaria Geral Administrativa,Departamento Administrativo,Motorista,NaN,26 ano(s),03/07/1995,01/10/2021,Pagamento Mensal,Ativo,148,6,1,4721.68,2333,2017,07,2017-07
4,000148,JOSE ROBERTO RIBEIRO,Secretaria Geral Administrativa,Departamento Administrativo,Motorista,NaN,26 ano(s),03/07/1995,01/10/2021,Adiantamento,Ativo,148,1,1,1216.33,1216,2017,07,2017-07


In [14]:
df.dtypes

matricula         object
nome              object
secretaria        object
local             object
cargo             object
escolaridade     float64
tempo             object
admissao          object
demissao          object
tipo_folha        object
condicao          object
id_matricula       int64
Tpf_Id             int64
Cnd_Id             int64
valor_bruto      float64
valor_liquido      int64
ano               object
mes               object
competencia       object
dtype: object

### Data Manipulation

In [15]:
df[df['cargo'].str.contains('INFO')]

,matricula,nome,secretaria,local,cargo,escolaridade,tempo,admissao,demissao,tipo_folha,condicao,id_matricula,Tpf_Id,Cnd_Id,valor_bruto,valor_liquido,ano,mes,competencia


In [16]:
# Separa os valores de matricula/digio, mes/ano 
df['matricula'] = df['matricula'].str[1:]
df['matricula']

0      00028
1      00028
2      00091
3      00148
4      00148
       ...  
452    01909
453    01910
454    01911
455    01912
456    01913
Name: matricula, Length: 30024, dtype: object

In [17]:
df['matric'] = df['matricula'].astype(float)
df

,matricula,nome,secretaria,local,cargo,escolaridade,tempo,admissao,demissao,tipo_folha,condicao,id_matricula,Tpf_Id,Cnd_Id,valor_bruto,valor_liquido,ano,mes,competencia,matric
0,00028,RAIMUNDO ANTONIO LUIZ,Secretaria Geral Legislativa,Departamento Financeiro,Chefe de Divisço,NaN,39 ano(s),28/01/1982,03/11/2021,Adiantamento,Ativo,28,1,1,3300.09,3300,2017,07,2017-07,28.0
1,00028,RAIMUNDO ANTONIO LUIZ,Secretaria Geral Legislativa,Departamento Financeiro,Chefe de Divisço,NaN,39 ano(s),28/01/1982,03/11/2021,Pagamento Mensal,Ativo,28,6,1,19416.31,3361,2017,07,2017-07,28.0
2,00091,PEDRO HIDEKI KOMURA,Vereadores,Inativos,Presidente da Camara,NaN,30 ano(s),01/01/1993,NaN,Pagamento Mensal,Sem Remunera‡Æo Fixa,91,6,2,12959.15,7840,2017,07,2017-07,91.0
3,00148,JOSE ROBERTO RIBEIRO,Secretaria Geral Administrativa,Departamento Administrativo,Motorista,NaN,26 ano(s),03/07/1995,01/10/2021,Pagamento Mensal,Ativo,148,6,1,4721.68,2333,2017,07,2017-07,148.0
4,00148,JOSE ROBERTO RIBEIRO,Secretaria Geral Administrativa,Departamento Administrativo,Motorista,NaN,26 ano(s),03/07/1995,01/10/2021,Adiantamento,Ativo,148,1,1,1216.33,1216,2017,07,2017-07,148.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,01909,MARLI VASCONCELOS GARCIA CAVALHEIRO,Secretaria Geral Legislativa,Departamento Financeiro,Assessor para Assuntos Politico-legislativos,NaN,3 mes(es),29/06/2023,NaN,Pagamento Mensal,Ativo,1909,6,1,6474.36,3227,2023,07,2023-07,1909.0
453,01910,EVERTON CARDOSO DE SOUSA,Secretaria Geral Legislativa,Departamento Financeiro,Assessor para Assuntos Politico-legislativos,NaN,2 mes(es),12/07/2023,NaN,Pagamento Mensal,Ativo,1910,6,1,3844.15,3414,2023,07,2023-07,1910.0
454,01911,VANESSA DOS SANTOS LUCAS PAULA,Secretaria Geral Legislativa,Departamento Financeiro,Assessor para Assuntos Politico-legislativos,NaN,2 mes(es),07/07/2023,NaN,Pagamento Mensal,Ativo,1911,6,1,4855.77,4028,2023,07,2023-07,1911.0
455,01912,PEDRO IVO VIEIRA ALVES DE SOUSA,Secretaria Geral Legislativa,Departamento Financeiro,Assessor para Assuntos Politico-legislativos,NaN,2 mes(es),24/07/2023,NaN,Pagamento Mensal,Ativo,1912,6,1,1416.27,1309,2023,07,2023-07,1912.0


### Normalização de cargos

In [18]:
df['cargo'] = df['cargo'].str.upper()
df['cargo_normalizado'] = df['cargo']

In [19]:
df['cargo_normalizado'] = df['cargo_normalizado'].replace(
    {'ASSESSOR ESPECIAL PARLAMENTAR'                  : 'ASSESSOR',
    'ASSESSOR PARA ASSUNTOS POLITICO-LEGISLATIVOS'    : 'ASSESSOR',
    'ASSESSOR DE CERIMONIAL'                          : 'ASSESSOR',
    'ASSESSOR LEGISLATIVO FINANCEIRO'                 : 'ASSESSOR',
    'ASSESSOR PARLAMENTAR DE GABINETE DA PRESIDENCIA' : 'ASSESSOR'}
)

{c: '' for c in df[['matricula', 'nome', 'cargo', 'cargo_normalizado']][df['cargo_normalizado'].str.upper().str.contains(r'^ASSES', regex=True)]['cargo_normalizado'].unique()}

{'ASSESSOR': ''}

In [20]:
df['cargo_normalizado'] = df['cargo_normalizado'].replace(
    {'PROCURADOR JURIDICO'      : 'PROCURADOR JURIDICO',
    'PROCURADOR JURIDICO CHEFE' : 'PROCURADOR JURIDICO'}
)

{c: '' for c in df[['matricula', 'nome', 'cargo', 'cargo_normalizado']][df['cargo_normalizado'].str.upper().str.contains(r'PROCUR', regex=True)]['cargo_normalizado'].unique()}

{'PROCURADOR JURIDICO': ''}

In [21]:
df['cargo_normalizado'] = df['cargo_normalizado'].replace(
    {'SECRETARIO GERAL LEGISLATIVO'   : 'SECRETARIO GERAL',
    'SECRETARIO GERAL ADMINISTRATIVO' : 'SECRETARIO GERAL'}
)

{c: '' for c in df[['matricula', 'nome', 'cargo', 'cargo_normalizado']][df['cargo_normalizado'].str.upper().str.contains(r'SECRET', regex=True)]['cargo_normalizado'].unique()}

{'SECRETARIO GERAL': ''}

In [22]:
df['cargo_normalizado'] = df['cargo_normalizado'].replace(
    {'CHEFE DE DIVISÇO'               : 'CHEFE',
    'CHEFE DA ASSESSORIA LEGISLATIVA' : 'CHEFE',
    'CHEFE DE GABINETE PARLAMENTAR'   : 'CHEFE'}
)

{c: '' for c in df[['matricula', 'nome', 'cargo', 'cargo_normalizado']][df['cargo_normalizado'].str.upper().str.contains(r'CHEFE', regex=True)]['cargo_normalizado'].unique()}

{'CHEFE': ''}

In [23]:
df['cargo_normalizado'] = (df['cargo_normalizado']
                           .str.replace('"', ' ', regex=False)
                           .str.replace('/', '-', regex=False)                           
                           )

In [24]:
df['comp'] = df['competencia']
df['comp']

0      2017-07
1      2017-07
2      2017-07
3      2017-07
4      2017-07
        ...   
452    2023-07
453    2023-07
454    2023-07
455    2023-07
456    2023-07
Name: comp, Length: 30024, dtype: object

In [25]:
df.dtypes

matricula             object
nome                  object
secretaria            object
local                 object
cargo                 object
escolaridade         float64
tempo                 object
admissao              object
demissao              object
tipo_folha            object
condicao              object
id_matricula           int64
Tpf_Id                 int64
Cnd_Id                 int64
valor_bruto          float64
valor_liquido          int64
ano                   object
mes                   object
competencia           object
matric               float64
cargo_normalizado     object
comp                  object
dtype: object

### Save output file into processed directory

Save a file in the processed directory that is cleaned properly. It will be read in and used later for further analysis.

Other options besides pickle include:
- feather
- msgpack
- parquet

In [26]:
# df.to_pickle(summary_file)
import contextlib
ano = int(max(df['ano'].unique()))
mes = int(max(df[df['ano'] == str(ano)]['mes'].unique()))
ano = f'{ano:0>4}'
mes = f'{mes:0>2}'

matriculas_ultimo_mes = list(df[(df['ano'] == ano) & (df['mes'] == mes)]['matricula'].unique())
print(len(matriculas_ultimo_mes))

configuracoes = ['ano', 'mes', 'cargo_normalizado']
for config in configuracoes:
    unique_values = df[config].unique()
    for value in unique_values:
        if config == 'cargo_normalizado':
            value = value.replace('"', ' ')
            value = value.replace("/", ' ')

        with contextlib.suppress(FileExistsError):
            destino = (Path.cwd() / 'data')
            destino.mkdir()
        with contextlib.suppress(FileExistsError):
            destino = (destino / 'processed')
            destino.mkdir()
        with contextlib.suppress(FileExistsError):
            destino = (destino / config)
            destino.mkdir()
            
        if config == 'matricula':           
            value = f'{value:0<5}'
            
            if value in matriculas_ultimo_mes:
                destino = destino / f"{value}.parquet"
                df[(df[config] == value)].to_parquet(destino)
                
        else:
            destino = destino / f"{value:0>2}.parquet"
            df[df[config] == value].to_parquet(destino)
                

231
